In [ ]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken cohere faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.5/178.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 468.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.9 MB/

# BabyAGI with LangChain

Task Creation  
Task Prioritization   
Execution

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [ ]:
!pip show langchain

Name: langchain
Version: 0.2.8
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


### Setting up BabyAGI

In [ ]:
!pip install -q langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chains.base import Chain

In [ ]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

In [ ]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()

# Initialize the vectorstore as empty
import faiss
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Chains

In [ ]:
class TaskCreationChain(LLMChain):
    """Chain to generates tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_creation_template = (
            "You are an task creation AI that uses the result of an execution agent"
            " to create new tasks with the following objective: {objective},"
            " The last completed task has the result: {result}."
            " This result was based on this task description: {task_description}."
            " These are incomplete tasks: {incomplete_tasks}."
            " Based on the result, create new tasks to be completed"
            " by the AI system that do not overlap with incomplete tasks."
            " Return the tasks as an array."
        )
        prompt = PromptTemplate(
            template=task_creation_template,
            input_variables=["result", "task_description", "incomplete_tasks", "objective"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [ ]:
class TaskPrioritizationChain(LLMChain):
    """Chain to prioritize tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
        """Get the response parser."""
        task_prioritization_template = (
            "You are an task prioritization AI tasked with cleaning the formatting of and reprioritizing"
            " the following tasks: {task_names}."
            " Consider the ultimate objective of your team: {objective}."
            " Do not remove any tasks. Return the result as a numbered list, like:"
            " #. First task"
            " #. Second task"
            " Start the task list with number {next_task_id}."
        )
        prompt = PromptTemplate(
            template=task_prioritization_template,
            input_variables=["task_names", "next_task_id", "objective"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

In [ ]:
# class ExecutionChain(LLMChain):
#     """Chain to execute tasks."""

#     @classmethod
#     def from_llm(cls, llm: BaseLLM, verbose: bool = True) -> LLMChain:
#         """Get the response parser."""
#         execution_template = (
#             "You are an AI who performs one task based on the following objective: {objective}."
#             " Take into account these previously completed tasks: {context}."
#             " Your task: {task}."
#             " Response:"
#         )
#         prompt = PromptTemplate(
#             template=execution_template,
#             input_variables=["objective", "context", "task"],
#         )
#         return cls(prompt=prompt, llm=llm, verbose=verbose)

#### With tools

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain
todo_prompt = PromptTemplate.from_template("You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}")
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name = "TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!"
    )
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context","agent_scratchpad"]
)

## BabyAGI Controller

In [ ]:
def get_next_task(task_creation_chain: LLMChain, result: Dict, task_description: str, task_list: List[str], objective: str) -> List[Dict]:
    """Get the next task."""
    incomplete_tasks = ", ".join(task_list)
    response = task_creation_chain.run(result=result, task_description=task_description, incomplete_tasks=incomplete_tasks, objective=objective)
    new_tasks = response.split('\n')
    return [{"task_name": task_name} for task_name in new_tasks if task_name.strip()]

In [ ]:
def prioritize_tasks(task_prioritization_chain: LLMChain, this_task_id: int, task_list: List[Dict], objective: str) -> List[Dict]:
    """Prioritize tasks."""
    task_names = [t["task_name"] for t in task_list]
    next_task_id = int(this_task_id) + 1
    response = task_prioritization_chain.run(task_names=task_names,
                                             next_task_id=next_task_id,
                                             objective=objective)
    new_tasks = response.split('\n')
    prioritized_task_list = []
    for task_string in new_tasks:
        if not task_string.strip():
            continue
        task_parts = task_string.strip().split(".", 1)
        if len(task_parts) == 2:
            task_id = task_parts[0].strip()
            task_name = task_parts[1].strip()
            prioritized_task_list.append({"task_id": task_id, "task_name": task_name})
    return prioritized_task_list

In [ ]:
def _get_top_tasks(vectorstore, query: str, k: int) -> List[str]:
    """Get the top k tasks based on the query."""
    results = vectorstore.similarity_search_with_score(query, k=k)
    if not results:
        return []
    sorted_results, _ = zip(*sorted(results, key=lambda x: x[1], reverse=True))
    return [str(item.metadata['task']) for item in sorted_results]

def execute_task(vectorstore, execution_chain: LLMChain, objective: str, task: str, k: int = 5) -> str:
    """Execute a task."""
    context = _get_top_tasks(vectorstore, query=objective, k=k)
    return execution_chain.run(objective=objective, context=context, task=task)

In [ ]:
# class BabyAGI(Chain, BaseModel):
#     """Controller model for the BabyAGI agent."""

#     task_list: deque = Field(default_factory=deque)
#     task_creation_chain: TaskCreationChain = Field(...)
#     task_prioritization_chain: TaskPrioritizationChain = Field(...)
#     execution_chain: ExecutionChain = Field(...)
#     task_id_counter: int = Field(1)
#     vectorstore: VectorStore = Field(init=False)
#     max_iterations: Optional[int] = None

#     class Config:
#         """Configuration for this pydantic object."""
#         arbitrary_types_allowed = True

#     def add_task(self, task: Dict):
#         self.task_list.append(task)

#     def print_task_list(self):
#         print("\033[95m\033[1m" + "\n*****TASK LIST*****\n" + "\033[0m\033[0m")
#         for t in self.task_list:
#             print(str(t["task_id"]) + ": " + t["task_name"])

#     def print_next_task(self, task: Dict):
#         print("\033[92m\033[1m" + "\n*****NEXT TASK*****\n" + "\033[0m\033[0m")
#         print(str(task["task_id"]) + ": " + task["task_name"])

#     def print_task_result(self, result: str):
#         print("\033[93m\033[1m" + "\n*****TASK RESULT*****\n" + "\033[0m\033[0m")
#         print(result)

#     @property
#     def input_keys(self) -> List[str]:
#         return ["objective"]

#     @property
#     def output_keys(self) -> List[str]:
#         return []

#     def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
#         """Run the agent."""
#         objective = inputs['objective']
#         first_task = inputs.get("first_task", "Make a todo list")
#         self.add_task({"task_id": 1, "task_name": first_task})
#         num_iters = 0
#         while True:
#             if self.task_list:
#                 self.print_task_list()

#                 # Step 1: Pull the first task
#                 task = self.task_list.popleft()
#                 self.print_next_task(task)

#                 # Step 2: Execute the task
#                 result = execute_task(
#                     self.vectorstore, self.execution_chain, objective, task["task_name"]
#                 )
#                 this_task_id = int(task["task_id"])
#                 self.print_task_result(result)

#                 # Step 3: Store the result in Pinecone
#                 result_id = f"result_{task['task_id']}"
#                 self.vectorstore.add_texts(
#                     texts=[result],
#                     metadatas=[{"task": task["task_name"]}],
#                     ids=[result_id],
#                 )

#                 # Step 4: Create new tasks and reprioritize task list
#                 new_tasks = get_next_task(
#                     self.task_creation_chain, result, task["task_name"], [t["task_name"] for t in self.task_list], objective
#                 )
#                 for new_task in new_tasks:
#                     self.task_id_counter += 1
#                     new_task.update({"task_id": self.task_id_counter})
#                     self.add_task(new_task)
#                 self.task_list = deque(
#                     prioritize_tasks(
#                         self.task_prioritization_chain, this_task_id, list(self.task_list), objective
#                     )
#                 )
#             num_iters += 1
#             if self.max_iterations is not None and num_iters == self.max_iterations:
#                 print("\033[91m\033[1m" + "\n*****TASK ENDING*****\n" + "\033[0m\033[0m")
#                 break
#         return {}

#     @classmethod
#     def from_llm(
#         cls,
#         llm: BaseLLM,
#         vectorstore: VectorStore,
#         verbose: bool = False,
#         **kwargs
#     ) -> "BabyAGI":
#         """Initialize the BabyAGI Controller."""
#         task_creation_chain = TaskCreationChain.from_llm(
#             llm, verbose=verbose
#         )
#         task_prioritization_chain = TaskPrioritizationChain.from_llm(
#             llm, verbose=verbose
#         )
#         execution_chain = ExecutionChain.from_llm(llm, verbose=verbose)
#         return cls(
#             task_creation_chain=task_creation_chain,
#             task_prioritization_chain=task_prioritization_chain,
#             execution_chain=execution_chain,
#             vectorstore=vectorstore,
#             **kwargs
#         )

In [ ]:
type(Chain)

pydantic.v1.main.ModelMetaclass

In [ ]:
type(BaseModel)

pydantic.v1.main.ModelMetaclass

In [ ]:
class BabyAGI(Chain, BaseModel):
    """Controller model for the BabyAGI agent."""

    task_list: deque = Field(default_factory=deque)
    task_creation_chain: TaskCreationChain = Field(...)
    task_prioritization_chain: TaskPrioritizationChain = Field(...)
    execution_chain: AgentExecutor = Field(...)
    task_id_counter: int = Field(1)
    vectorstore: VectorStore = Field(init=False)
    max_iterations: Optional[int] = None

    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True

    def add_task(self, task: Dict):
        self.task_list.append(task)

    def print_task_list(self):
        print("\033[95m\033[1m" + "\n*****TASK LIST*****\n" + "\033[0m\033[0m")
        for t in self.task_list:
            print(str(t["task_id"]) + ": " + t["task_name"])

    def print_next_task(self, task: Dict):
        print("\033[92m\033[1m" + "\n*****NEXT TASK*****\n" + "\033[0m\033[0m")
        print(str(task["task_id"]) + ": " + task["task_name"])

    def print_task_result(self, result: str):
        print("\033[93m\033[1m" + "\n*****TASK RESULT*****\n" + "\033[0m\033[0m")
        print(result)

    @property
    def input_keys(self) -> List[str]:
        return ["objective"]

    @property
    def output_keys(self) -> List[str]:
        return []

    def _call(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Run the agent."""
        objective = inputs['objective']
        first_task = inputs.get("first_task", "Make a todo list")
        self.add_task({"task_id": 1, "task_name": first_task})
        num_iters = 0
        while True:
            if self.task_list:
                self.print_task_list()

                # Step 1: Pull the first task
                task = self.task_list.popleft()
                self.print_next_task(task)

                # Step 2: Execute the task
                result = execute_task(
                    self.vectorstore, self.execution_chain, objective, task["task_name"]
                )
                this_task_id = int(task["task_id"])
                self.print_task_result(result)

                # Step 3: Store the result in Pinecone
                result_id = f"result_{task['task_id']}"
                self.vectorstore.add_texts(
                    texts=[result],
                    metadatas=[{"task": task["task_name"]}],
                    ids=[result_id],
                )

                # Step 4: Create new tasks and reprioritize task list
                new_tasks = get_next_task(
                    self.task_creation_chain, result, task["task_name"], [t["task_name"] for t in self.task_list], objective
                )
                for new_task in new_tasks:
                    self.task_id_counter += 1
                    new_task.update({"task_id": self.task_id_counter})
                    self.add_task(new_task)
                self.task_list = deque(
                    prioritize_tasks(
                        self.task_prioritization_chain, this_task_id, list(self.task_list), objective
                    )
                )
            num_iters += 1
            if self.max_iterations is not None and num_iters == self.max_iterations:
                print("\033[91m\033[1m" + "\n*****TASK ENDING*****\n" + "\033[0m\033[0m")
                break
        return {}

    @classmethod
    def from_llm(
        cls,
        llm: BaseLLM,
        vectorstore: VectorStore,
        verbose: bool = False,
        **kwargs
    ) -> "BabyAGI":
        """Initialize the BabyAGI Controller."""
        task_creation_chain = TaskCreationChain.from_llm(
            llm, verbose=verbose
        )
        task_prioritization_chain = TaskPrioritizationChain.from_llm(
            llm, verbose=verbose
        )
        llm_chain = LLMChain(llm=llm, prompt=prompt)
        tool_names = [tool.name for tool in tools]
        agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
        agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
        return cls(
            task_creation_chain=task_creation_chain,
            task_prioritization_chain=task_prioritization_chain,
            execution_chain=agent_executor,
            vectorstore=vectorstore,
            **kwargs
        )

## Run baby

In [ ]:

OBJECTIVE = "Find the best way for learning Software Architecture"

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
# Logging of LLMChains
verbose=False
# If None, will keep on going forever
max_iterations: Optional[int] = 7
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    verbose=verbose,
    max_iterations=max_iterations
)

In [ ]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should consider the objective and any relevant information to create a useful todo list
Action: TODO
Action Input: objective, callbacks, tags, metadata
Observation: 


1. Define the objective: The first step in creating a todo list is to clearly define the objective. This will help guide all the other tasks and ensure that they are aligned with the overall goal.

2. Identify potential callbacks: Callbacks are important for achieving the objective as they allow for feedback and adjustments along the way. Brainstorm and list out potential callbacks that could be useful for this objective.

3. Assign tags: Tags are a great way to organize and categorize tasks. Assign relevant tags to each task on the todo list to make it easier to track and prioritize.

4. Gather necessary metadata: Metadata refers to any additional information or data that may be needed t

ValueError: Tried to add ids that already exist: {'result_1'}

In [ ]:
OBJECTIVE = "Find the best books for learning AI Engineering. Give me its URLs and finish your task there"
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should gather information about the objective and think about what tasks need to be completed
Action: TODO
Action Input: objective to create a todo list for
Observation:  a successful job search

1. Research and identify potential job opportunities in your desired field or industry.
2. Update and tailor your resume and cover letter to fit each job application.
3. Network and reach out to contacts in your industry for potential job leads or referrals.
4. Utilize online job search engines and company websites to search for job openings.
5. Create a list of target companies and research their company culture, values, and job openings.
6. Set a daily or weekly goal for the number of job applications to submit.
7. Prepare for interviews by researching common interview questions and practicing your responses.
8. Update your LinkedIn profile and utilize the pl

KeyboardInterrupt: 

In [ ]:
OBJECTIVE = "Write a weather report for Melbourne Australia today in Celsius"
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should think about what tasks I need to complete
Action: TODO
Action Input: Write a weather report for Melbourne Australia today in Celsius
Observation: 

1. Research the current weather conditions in Melbourne, Australia.
2. Gather information on the temperature, humidity, wind speed, and any other relevant weather data.
3. Convert the temperature from Fahrenheit to Celsius.
4. Write an introduction to the weather report, including the date and location.
5. Describe the current temperature in Celsius and compare it to the average temperature for this time of year.
6. Discuss the humidity levels and how it may affect the perceived temperature.
7. Mention any significant weather events or patterns that may be occurring in Melbourne today.
8. Provide information on the wind speed and direction.
9. Include a forecast for the rest of the day, including any 

KeyboardInterrupt: 